In [5]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.cuore_thermalmodeldata

from tqdm import tqdm
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pylab import *

from sklearn.grid_search import ParameterGrid

from eventGenerator import *

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Set Initial Parameters and Constants

In [2]:
alpha = [2.7e-8,9.9e-9,1e-11,9.14e3,4.28e-7,0]
beta = [3,4.37,2.4,3,2.4,2,1]
k = [2.34e-3,1.4,3.2e-5,1.3e-3,3.2e-5,2.5e-5,4e-7]

R0 = 3.2
s = .015
Rl = 6e10
T0 = 5.2
gamma = 0.5
#Vb = 5
Cp = 5e-10

V_Bias = [0.4302,0.8579,1.29,1.72,2.149,2.576,3.003,3.432,3.859,4.291,4.718,5.147,5.577,6.006,6.437,6.865,7.295,7.72,8.15,8.58,9.008]
V_Bol = [0.001958,0.003674,0.005138,0.006339,0.007354,0.0082,0.00892,0.009538,0.01009,0.01055,0.01097,0.01134,0.01165,0.01193,0.0122,0.01243,0.01264,0.01283,0.013,0.01316,0.01333]
I_Bol = [7.14E-12,1.42E-11,2.14E-11,2.86E-11,3.57E-11,4.28E-11,4.99E-11,5.70E-11,6.42E-11,7.13E-11,7.84E-11,8.56E-11,9.28E-11,9.99E-11,1.07E-10,1.14E-10,1.21E-10,1.29E-10,1.36E-10,1.43E-10,1.50E-10]

## Procedure to find best parameters

1. Choose large proportion ~10%
2. Generate N points and get chi^2 vars for each
3. Return parameters of points that has lowest chi^2, and set as new parameters
4. Lower proportion by 2% and iterate
5. Get last value

### Generate Values

In [3]:
def arrayRandomVars(array,prop):
    return np.random.uniform(np.multiply(array,(1-prop)), np.multiply(array,(1+prop)))

num = 5
prop = 0.2

bestChi2 = 10000
bestAlpha, bestBeta, bestK = [],[],[]

for j in tqdm(range(1,num)):
    
    V_Data, I_Data = [],[]
    
    alpha = arrayRandomVars(alpha,prop)
    beta = arrayRandomVars(beta,prop)
    k = arrayRandomVars(k,prop)
    
    for b in tqdm(V_Bias):
        v,i = getTemps(alpha, beta, k, s, R0, Rl, T0, gamma, b, Cp)
        V_Data.append(v)
        I_Data.append(i)
        
    v_chi = getchi2(V_Bol,V_Data)
    
    if v_chi < bestChi2:
        bestChi2 = v_chi
        bestAlpha = alpha
        bestBeta = beta
        bestK = k
        
print bestAlpha, bestBeta, bestK

  0%|          | 0/21 [00:00<?, ?it/s]


  5%|▍         | 1/21 [11:13<3:44:37, 673.89s/it]

99.98% | (13:06|00:00) 



 10%|▉         | 2/21 [22:04<3:29:44, 662.32s/it]

99.98% | (13:22|00:00) 



 14%|█▍        | 3/21 [32:52<3:17:17, 657.65s/it]

99.98% | (13:21|00:00) 


KeyboardInterrupt: 

In [2]:

V_Bias = [0.4302,0.8579,1.29,1.72,2.149,2.576,3.003,3.432,3.859,4.291,4.718,5.147,5.577,6.006,6.437,6.865,7.295,7.72,8.15,8.58,9.008]
V_Bol = [0.001958,0.003674,0.005138,0.006339,0.007354,0.0082,0.00892,0.009538,0.01009,0.01055,0.01097,0.01134,0.01165,0.01193,0.0122,0.01243,0.01264,0.01283,0.013,0.01316,0.01333]
I_Bol = [7.14E-12,1.42E-11,2.14E-11,2.86E-11,3.57E-11,4.28E-11,4.99E-11,5.70E-11,6.42E-11,7.13E-11,7.84E-11,8.56E-11,9.28E-11,9.99E-11,1.07E-10,1.14E-10,1.21E-10,1.29E-10,1.36E-10,1.43E-10,1.50E-10]


In [8]:
def getchi2(test, data):
    return sum(abs(np.divide(np.power((np.subtract(data,test)),2),test)))

print getchi2(V_Bias,V_Bol)

98.6970910878
